<a href="https://colab.research.google.com/github/rahmanziaur/DTClassifierTest/blob/main/1_CUML_100%20Column%20and%203M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [2]:
!nvidia-smi

Thu Oct 27 12:44:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. BlazingSQL
  1. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [3]:
!pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.5 MB/s 


In [4]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 300, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 300 (delta 74), reused 99 (delta 55), pack-reused 171
Receiving objects: 100% (300/300), 87.58 KiB | 7.96 MiB/s, done.
Resolving deltas: 100% (136/136), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu bionic InRelease [20.8 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,217 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:28
🔁 Restarting kernel...


In [1]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

# RAPIDS is now installed on Colab.  You can copy your code into the cells below.  Enjoy!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy dataset 30 times
import csv

N = 30

# /content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv

# dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv')

with open('', 'r') as infile,\
         open('/content/drive/MyDrive/Colab Notebooks/Zia Sir/CUML/abc_less_feature30.csv', 'w') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    writer.writerow(next(reader))    # reads header line and writes it to output file
    data = [row for row in reader]    # reads the rest of the input file
    for i in range(N):
        writer.writerows(data)

In [4]:
# Copy dataset 30 times
import csv

N = 30

# /content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv

# dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv')

with open('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv', 'r') as infile,\
         open('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature30.csv', 'w') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    writer.writerow(next(reader))    # reads header line and writes it to output file
    data = [row for row in reader]    # reads the rest of the input file
    for i in range(N):
        writer.writerows(data)

In [6]:
import timeit
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Change the classifier to test the speed of both sklearn and cuml
# from sklearn.ensemble import RandomForestClassifier
from cuml.ensemble import RandomForestClassifier


from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
#warnings.filterwarnings('always') 

# Copy the path of the .csv file produced earlier
# dataset=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Zia Sir/20-09-22/abc_less_feature.csv')

# 100 k data
#dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv')

# 100 k data
dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature30.csv')

print(list(dataset.columns))
feature_cols= list(dataset.columns[:-1])

target_cols=list(dataset.columns[-1:])
#target_cols

#split dataset in features and target variable
X = dataset.drop('readmitted', axis=1) # Features
y = dataset['readmitted'] # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
# Check the shape of all of these
# print("X_train shape is : ", X_train.shape)
# print("X_test shape  is : ", X_test.shape)
# print("y_train shape is : ", y_train.shape)
# print("y_test shape is  : ", y_test.shape)

#Calculate start time
start = timeit.default_timer()

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)
# clf = DecisionTreeClassifier(max_depth = 2, random_state = 0)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

#Calculate Stop time
stop = timeit.default_timer()
train_time= stop - start

#Calculate start time
start = timeit.default_timer()

# Predict the model
y_pred=clf.predict(X_test)

#Calculate Stop time
stop = timeit.default_timer()
test_time= stop - start

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

['encounter_id', 'patient_nbr', 'race', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'change', 'diabetesMed', 'service_utilization', 'numchange', 'readmitted']
[[813706      0]
 [ 75859  26329]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.96    813706
           1       1.00      0.26      0.41    102188

    accuracy                           0.92    915894
   macro avg       0.96      0.63      0.68    915894
weighted avg       0.92      0.92      0.89    915894



In [8]:
import timeit
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Change the classifier to test the speed of both sklearn and cuml
# from sklearn.ensemble import RandomForestClassifier
from cuml.ensemble import RandomForestClassifier


from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
#warnings.filterwarnings('always') 

# Copy the path of the .csv file produced earlier
# dataset=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Zia Sir/20-09-22/abc_less_feature.csv')
#dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv')

# 100 k data
#dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature (2).csv')

# 100 k data
dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/abc_less_feature30.csv')

print(list(dataset.columns))
feature_cols= list(dataset.columns[:-1])

target_cols=list(dataset.columns[-1:])
#target_cols

#split dataset in features and target variable
X = dataset.drop('readmitted', axis=1) # Features
y = dataset['readmitted'] # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
# Check the shape of all of these
# print("X_train shape is : ", X_train.shape)
# print("X_test shape  is : ", X_test.shape)
# print("y_train shape is : ", y_train.shape)
# print("y_test shape is  : ", y_test.shape)

#Calculate start time
start = timeit.default_timer()

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)
# clf = DecisionTreeClassifier(max_depth = 2, random_state = 0)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

#Calculate Stop time
stop = timeit.default_timer()
train_time= stop - start

#Calculate start time
start = timeit.default_timer()

# Predict the model
y_pred=clf.predict(X_test)

#Calculate Stop time
stop = timeit.default_timer()
test_time= stop - start

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

['encounter_id', 'patient_nbr', 'race', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'change', 'diabetesMed', 'service_utilization', 'numchange', 'readmitted']
[[813706      0]
 [ 75859  26329]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.96    813706
           1       1.00      0.26      0.41    102188

    accuracy                           0.92    915894
   macro avg       0.96      0.63      0.68    915894
weighted avg       0.92      0.92      0.89    915894



In [9]:
import timeit
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Change the classifier to test the speed of both sklearn and cuml
# from sklearn.ensemble import RandomForestClassifier
from cuml.ensemble import RandomForestClassifier


from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
#warnings.filterwarnings('always') 

# Copy the path of the .csv file produced earlier
dataset=pd.read_csv('/content/drive/MyDrive/0_ColabDatasets/1_HealthDMTool_ 100_column.csv')
# dataset=pd.read_csv('https://github.com/mustain-billah/healthdmtool/blob/6d63e39aa759b63befd3f3889a25f10a96a4e00f/abc_less_feature%20(2).csv')
# dataset=pd.read_csv('https://github.com/mustain-billah/healthdmtool/blob/0131e11571328f61566c3637f5839e06a6d38f08/HealthDMTool_%20100_column.csv')


print(list(dataset.columns))
feature_cols= list(dataset.columns[:-1])

target_cols=list(dataset.columns[-1:])
#target_cols

#split dataset in features and target variable
X = dataset.drop('readmitted', axis=1) # Features
y = dataset['readmitted'] # Target variable

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
# Check the shape of all of these
# print("X_train shape is : ", X_train.shape)
# print("X_test shape  is : ", X_test.shape)
# print("y_train shape is : ", y_train.shape)
# print("y_test shape is  : ", y_test.shape)

#Calculate start time
start = timeit.default_timer()

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)
# clf = DecisionTreeClassifier(max_depth = 2, random_state = 0)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

#Calculate Stop time
stop = timeit.default_timer()
train_time= stop - start

#Calculate start time
start = timeit.default_timer()

# Predict the model
y_pred=clf.predict(X_test)

#Calculate Stop time
stop = timeit.default_timer()
test_time= stop - start

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

['encounter_id', 'patient_nbr', 'race', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'change', 'diabetesMed', 'service_utilization', 'numchange', 'encounter_id.1', 'patient_nbr.1', 'race.1', 'age.1', 'admission_type_id.1', 'discharge_disposition_id.1', 'admission_source_id.1', 'time_in_hospital.1', 'num_lab_procedures.1', 'num_procedures.1', 'num_medications.1', 'number_diagnoses.1', 'max_glu_serum.1', 'A1Cresult.1', 'change.1', 'diabetesMed.1', 'service_utilization.1', 'numchange.1', 'encounter_id.2', 'patient_nbr.2', 'race.2', 'age.2', 'admission_type_id.2', 'discharge_disposition_id.2', 'admission_source_id.2', 'time_in_hospital.2', 'num_lab_procedures.2', 'num_procedures.2', 'num_medications.2', 'number_diagnoses.2', 'max_glu_serum.2', 'A1Cresult.2', 'change.2', 'diabetesMed.2', 'service_utilization.2', 'numchange.2', 'en

In [10]:
train_time

1.0290237640001578

In [11]:
test_time

3.9814098400001967

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-contrib

In [12]:
# This code is for comparing the speed of sklearn and cuml
from sklearn import datasets
import numpy as np
X, y  = datasets.make_classification(n_samples=40000)
X = X.astype(np.float32)
y = y.astype(np.float32)
def train_data(model, X=X, y=y):
    clf = model
    clf.fit(X, y)
from sklearn.svm import SVC 
from cuml.svm import SVC as SVC_gpu

clf_svc = SVC(kernel='poly', degree=2, gamma='auto', C=1)
sklearn_time_svc = %timeit -o train_data(clf_svc)

clf_svc = SVC_gpu(kernel='poly', degree=2, gamma='auto', C=1)
cuml_time_svc = %timeit -o train_data(clf_svc)

print(f"""Average time of sklearn's {clf_svc.__class__.__name__}""", sklearn_time_svc.average, 's')
print(f"""Average time of cuml's {clf_svc.__class__.__name__}""", cuml_time_svc.average, 's')

print('Ratio between sklearn and cuml is', sklearn_time_svc.average/cuml_time_svc.average)

1min 12s ± 458 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.91 s ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Average time of sklearn's SVC 72.62632194728569 s
Average time of cuml's SVC 2.9128898112857575 s
Ratio between sklearn and cuml is 24.9327391876963
